# Entrenamiento y Empaquetado de un Modelo de Machine Learning para Predicción de Matrícula

## Paso 1: Cargar y explorar los datos

In [14]:
import pandas as pd

# Cargar dataset
df = pd.read_csv("cursos_matricula.csv")

# Mostrar primeras filas
df.head()

,edad,genero,nivel_educativo,ingresos_mensuales,ocupacion,interes_tema,uso_tecnologia,horas_disponibles,promociones_recibidas,matricula
0,56,M,Universitario,995.0,Desempleado,2,4,0,1,0
1,46,M,Universitario,2172.0,Independiente,1,5,17,0,0
2,32,F,Universitario,797.0,Independiente,4,2,19,1,0
3,60,F,Postgrado,2034.0,Independiente,5,5,5,1,1
4,25,M,Universitario,872.0,Empleado,2,2,19,1,0


## Paso 2: Codificar variables categóricas

In [15]:
from sklearn.preprocessing import LabelEncoder

# Copia del dataframe para entrenamiento
df_modelo = df.copy()

# Codificación
encoders = {}
for col in ['genero', 'nivel_educativo', 'ocupacion']:
    le = LabelEncoder()
    df_modelo[col] = le.fit_transform(df_modelo[col])
    encoders[col] = le

# Mostrar datos codificados
df_modelo.head()

,edad,genero,nivel_educativo,ingresos_mensuales,ocupacion,interes_tema,uso_tecnologia,horas_disponibles,promociones_recibidas,matricula
0,56,1,4,995.0,0,2,4,0,1,0
1,46,1,4,2172.0,3,1,5,17,0,0
2,32,0,4,797.0,3,4,2,19,1,0
3,60,0,0,2034.0,3,5,5,5,1,1
4,25,1,4,872.0,1,2,2,19,1,0


## Paso 3: Separar variables predictoras (X) y variable objetivo (y)

In [16]:
X = df_modelo.drop(columns=['matricula'])
y = df_modelo['matricula']

print("Columnas predictoras (X):", list(X.columns))
print("Variable objetivo (y): matricula")

Columnas predictoras (X): ['edad', 'genero', 'nivel_educativo', 'ingresos_mensuales', 'ocupacion', 'interes_tema', 'uso_tecnologia', 'horas_disponibles', 'promociones_recibidas']
Variable objetivo (y): matricula


## Paso 4: Entrenar el modelo

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Separar en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
modelo = RandomForestClassifier(random_state=42)
modelo.fit(X_train, y_train)

# Ver resumen del modelo
print(modelo)
print("Importancia de las variables:", modelo.feature_importances_)

RandomForestClassifier(random_state=42)
Importancia de las variables: [0.0270444  0.00583298 0.06948657 0.03049507 0.00976301 0.30957637
 0.19252829 0.16575917 0.18951415]


## Paso 5: Guardar el modelo entrenado y los codificadores

In [20]:
import joblib

# Guardar modelo
joblib.dump(modelo, "modelo_entrenado.pkl")

# Guardar codificadores
joblib.dump(encoders, "encoders.pkl")

print("✅ Modelo y encoders guardados exitosamente.")

✅ Modelo y encoders guardados exitosamente.


In [21]:
import joblib

encoders = joblib.load("encoders.pkl")
print("Claves disponibles en el archivo:", list(encoders.keys()))


Claves disponibles en el archivo: ['genero', 'nivel_educativo', 'ocupacion']


## Paso 6: ¿Qué se hace luego?

Este modelo ya ha aprendido cómo predecir, pero…
Solo vive en la memoria de Python!

Con el modelo entrenado y guardado:
- Puedes cargarlo en una app Flask para hacer predicciones en tiempo real.
- No necesitas el archivo .csv para hacer predicciones, solo los `.pkl`.
- Puedes compartir el modelo con otros sin necesidad de compartir tu código de entrenamiento.

Para usarlo:

```python
import joblib
modelo = joblib.load("modelo_entrenado.pkl")
encoders = joblib.load("encoders.pkl")
```


## Paso 7: Evaluación del modelo con métricas

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Predicciones
y_pred = modelo.predict(X_test)

# Métricas
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))


Matriz de confusión:
[[542   0]
 [  0  58]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       542
           1       1.00      1.00      1.00        58

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


## Paso 8: Comparar múltiples modelos de clasificación

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Lista de modelos
modelos = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'DecisionTree': DecisionTreeClassifier(),
    'SVM': SVC()
}

# Evaluar cada modelo
for nombre, modelo_clasificador in modelos.items():
    modelo_clasificador.fit(X_train, y_train)
    pred = modelo_clasificador.predict(X_test)
    acc = accuracy_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print(f"{nombre}: Accuracy={acc:.4f}, F1 Score={f1:.4f}")


RandomForest: Accuracy=1.0000, F1 Score=1.0000
LogisticRegression: Accuracy=0.9450, F1 Score=0.6796
DecisionTree: Accuracy=1.0000, F1 Score=1.0000
SVM: Accuracy=0.9033, F1 Score=0.0000


C:\Users\User\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Paso 9: Explicación de métricas, épocas y desbalanceo de clases

### 📊 ¿Qué métricas se usan para comparar modelos?

| Métrica     | ¿Cuándo usarla? |
|-------------|-----------------|
| Accuracy    | Si las clases están balanceadas |
| Precision   | Si es más importante evitar falsos positivos |
| Recall      | Si es más importante evitar falsos negativos |
| F1-score    | Si necesitas equilibrio entre precision y recall |
| AUC-ROC     | Para comparar clasificación probabilística en todos los umbrales |

Estas métricas se aplican **siempre sobre los datos de prueba (`X_test`, `y_test`)**, nunca sobre los datos de entrenamiento.

```python
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))
```


### 🔁 ¿Qué son las épocas de entrenamiento?

Una **época** es una pasada completa del modelo sobre todos los datos de entrenamiento.

- Se usa principalmente en **redes neuronales** y modelos que aprenden de manera progresiva (como `SGDClassifier`, `MLPClassifier`, etc.).
- En modelos como RandomForest, LogisticRegression, o DecisionTree **no se usan épocas directamente**.

```python
# Ejemplo con redes neuronales
from sklearn.neural_network import MLPClassifier
modelo = MLPClassifier(max_iter=100)  # 100 épocas
```


### ⚠️ ¿Qué es el desbalanceo de clases?

Ocurre cuando una clase aparece mucho más que otra. Por ejemplo:

```python
df['matricula'].value_counts(normalize=True)
```

Si ves algo como:

```
0    0.90
1    0.10
```

Eso significa que el modelo puede predecir siempre "0" y tener 90% de accuracy, sin aprender nada útil.

### ¿Cómo manejarlo?
- Usa métricas como **Recall** o **F1-Score**.
- Aplica técnicas como:
  - `class_weight='balanced'` en algunos modelos
  - `SMOTE` para sobremuestrear la clase minoritaria
  - Submuestreo de la clase mayoritaria

```python
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight='balanced')
```


In [25]:
df['matricula'].value_counts(normalize=True)

matricula
0    0.902333
1    0.097667
Name: proportion, dtype: float64

In [24]:
import joblib

# Intentamos cargar el modelo y los encoders con joblib
try:
    modelo = joblib.load('modelo_entrenado.pkl')
    encoders = joblib.load('encoders.pkl')
    print("✅ Modelo y encoders cargados con éxito.")
except Exception as e:
    print("❌ Error al cargar los archivos:")
    print(e)

✅ Modelo y encoders cargados con éxito.


In [ ]:
import streamlit as st
import numpy as np

def predecir_matricula(edad, genero, nivel_educativo, ingresos_mensuales,
                       ocupacion, interes_tema, uso_tecnologia, horas_disponibles,
                       promociones_recibidas):
    try:
        # Codificar variables categóricas con LabelEncoder
        genero_encoded = encoders['genero'].transform([genero])[0]
        nivel_encoded = encoders['nivel_educativo'].transform([nivel_educativo])[0]
        ocupacion_encoded = encoders['ocupacion'].transform([ocupacion])[0]

        # Construir vector en el orden correcto
        X = [[
            edad,
            genero_encoded,
            nivel_encoded,
            ingresos_mensuales,
            ocupacion_encoded,
            interes_tema,
            uso_tecnologia,
            horas_disponibles,
            promociones_recibidas
        ]]

        # Predicción
        pred = modelo.predict(X)[0]
        return f"✅ Predicción: {pred}"

    except Exception as e:
        return f"❌ Error en la predicción: {e}"


# Configuración de la página
st.set_page_config(
    page_title="Predicción de Matrícula",
    page_icon="🎓",
    layout="wide"
)

# Título y descripción
st.title("🎓 Predicción de Matrícula")
st.markdown("Ingresa los datos del estudiante para predecir si se matriculará (0 = No, 1 = Sí).")

# Obtener listas de clases desde los LabelEncoders
generos = encoders['genero'].classes_.tolist()
niveles_educativos = encoders['nivel_educativo'].classes_.tolist()
ocupaciones = encoders['ocupacion'].classes_.tolist()

# Crear formulario con columnas para mejor organización
with st.form("prediction_form"):
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("Datos Personales")
        edad = st.number_input(
            "Edad",
            min_value=0,
            max_value=100,
            value=25,
            step=1
        )
        
        genero = st.selectbox(
            "Género",
            options=generos,
            index=0
        )
        
        nivel_educativo = st.selectbox(
            "Nivel educativo",
            options=niveles_educativos,
            index=0
        )
        
        ingresos_mensuales = st.number_input(
            "Ingresos mensuales",
            min_value=0.0,
            value=1000.0,
            step=100.0,
            format="%.2f"
        )
        
        ocupacion = st.selectbox(
            "Ocupación",
            options=ocupaciones,
            index=0
        )
    
    with col2:
        st.subheader("Preferencias y Disponibilidad")
        interes_tema = st.slider(
            "Interés en el tema",
            min_value=0,
            max_value=5,
            value=3,
            step=1
        )
        
        uso_tecnologia = st.slider(
            "Uso de tecnología",
            min_value=0,
            max_value=5,
            value=3,
            step=1
        )
        
        horas_disponibles = st.slider(
            "Horas disponibles",
            min_value=0,
            max_value=24,
            value=8,
            step=1
        )
        
        promociones_recibidas = st.slider(
            "Promociones recibidas",
            min_value=0,
            max_value=20,
            value=5,
            step=1
        )
    
    # Botón de predicción
    submit_button = st.form_submit_button(
        "🔮 Realizar Predicción",
        use_container_width=True
    )

# Procesar predicción cuando se envía el formulario
if submit_button:
    with st.spinner('Realizando predicción...'):
        resultado = predecir_matricula(
            edad, genero, nivel_educativo, ingresos_mensuales,
            ocupacion, interes_tema, uso_tecnologia, horas_disponibles,
            promociones_recibidas
        )
    
    # Mostrar resultado
    if "✅" in resultado:
        st.success(resultado)
        # Interpretar el resultado
        pred_value = resultado.split(": ")[1]
        if pred_value == "1":
            st.balloons()
            st.info("🎉 El estudiante tiene alta probabilidad de matricularse")
        else:
            st.info("📊 El estudiante tiene baja probabilidad de matricularse")
    else:
        st.error(resultado)

# Información adicional
with st.expander("ℹ️ Información sobre los parámetros"):
    st.markdown("""
    **Descripción de los parámetros:**
    
    - **Edad**: Edad del estudiante en años
    - **Género**: Género del estudiante
    - **Nivel educativo**: Nivel de educación completado
    - **Ingresos mensuales**: Ingresos económicos mensuales
    - **Ocupación**: Ocupación actual del estudiante
    - **Interés en el tema**: Nivel de interés (0-5, donde 5 es muy alto)
    - **Uso de tecnología**: Familiaridad con tecnología (0-5)
    - **Horas disponibles**: Horas disponibles para estudiar por día
    - **Promociones recibidas**: Número de promociones/ofertas recibidas
    """)

# Pie de página
st.markdown("---")
st.markdown("*Desarrollado con Streamlit* 🚀")